In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [113]:
#Loading datasets
crime_df = pd.read_csv('Chicago_Crime_Data_Cleaned.csv')
police_sentiment_df = pd.read_csv('Police_Sentiment_Scores_20231109.csv')
grocery_stores_df = pd.read_csv('Grocery_Stores_-_2013_20231109.csv')
train_ridership_df = pd.read_csv('CTA_-_Ridership_-__L__Station_Entries_-_Monthly_Day-Type_Averages___Totals_20231109.csv')
iucr_codes_df = pd.read_csv('Chicago_Police_Department_-_Illinois_Uniform_Crime_Reporting__IUCR__Codes.csv')
census_df = pd.read_csv('Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012_20231109.csv')
housing_df = pd.read_csv('Affordable_Rental_Housing_Developments_20231109.csv')
bus_df = pd.read_csv('CTA_-_Ridership_-_Bus_Routes_-_Monthly_Day-Type_Averages___Totals.csv') # New Dataset. Bus routes data


In [88]:
crime_df.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,5741943,HN549294,2007-08-25 09:22:18,074XX N ROGERS AVE,0560,ASSAULT,SIMPLE,OTHER,False,False,2422,24.0,49.0,1.0,08A,NaN,NaN,2007,2015-08-17 15:03:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930689,HH109118,2002-01-05 21:24:00,007XX E 103 ST,0820,THEFT,$500 AND UNDER,GAS STATION,True,False,512,5.0,NaN,NaN,06,NaN,NaN,2002,2016-02-04 06:33:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13203321,JG415333,2023-09-06 17:00:00,002XX N Wells st,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,122,1.0,42.0,32.0,14,1174694.0,1901831.0,2023,2023-11-04 15:40:18,41.886018,-87.633938,"(41.886018055, -87.633937881)",22.0,14309.0,38.0,92.0,36.0,46.0,22.0,79.0
3,13210088,JG423627,2023-08-31 12:00:00,023XX W JACKSON BLVD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,STREET,False,False,1225,12.0,27.0,28.0,11,1160870.0,1898642.0,2023,2023-09-16 15:41:56,41.877565,-87.684791,"(41.877565108, -87.68479102)",48.0,21184.0,29.0,766.0,46.0,28.0,15.0,139.0
4,13210004,JG422532,2023-07-24 21:45:00,073XX S JEFFERY BLVD,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,APARTMENT,False,False,333,3.0,7.0,43.0,02,1190812.0,1856743.0,2023,2023-09-16 15:41:56,41.761919,-87.576209,"(41.7619185, -87.576209245)",32.0,22538.0,39.0,419.0,37.0,24.0,18.0,89.0


In [90]:
#Looking at all the location descriptors and seeing which are most specific

# Counting unique values in the specified columns, including Zip Codes
unique_beats = crime_df['Beat'].nunique()
unique_districts = crime_df['District'].nunique()
unique_ward = crime_df['Ward'].nunique()
unique_community_area = crime_df['Community Area'].nunique()
unique_historical_wards = crime_df['Historical Wards 2003-2015'].nunique()
unique_zip_codes = crime_df['Zip Codes'].nunique()
unique_community_areas = crime_df['Community Areas'].nunique()
unique_census_tracts = crime_df['Census Tracts'].nunique()
unique_wards = crime_df['Wards'].nunique()
unique_boundaries = crime_df['Boundaries - ZIP Codes'].nunique()
unique_police_districts = crime_df['Police Districts'].nunique()
unique_police_beats = crime_df['Police Beats'].nunique()

# Displaying the number of unique values
print("Number of unique Beats:", unique_beats)
print("Number of unique Districts:", unique_districts)
print("Number of unique Ward:", unique_ward)
print("Number of unique Community Area:", unique_community_area)
print("Number of unique Historical Wards 2003-2015:", unique_historical_wards)
print("Number of unique Zip Codes:", unique_zip_codes)
print("Number of unique Community Areas:", unique_community_areas)
print("Number of unique Census Tracts:", unique_census_tracts)
print("Number of unique Wards:", unique_wards)
print("Number of unique Boundaires:", unique_boundaries)
print("Number of unique Police Districts:", unique_police_districts)
print("Number of unique Police Beats:", unique_police_beats)



Number of unique Beats: 305
Number of unique Districts: 24
Number of unique Ward: 50
Number of unique Community Area: 78
Number of unique Historical Wards 2003-2015: 53
Number of unique Zip Codes: 83
Number of unique Community Areas: 77
Number of unique Census Tracts: 801
Number of unique Wards: 50
Number of unique Boundaires: 61
Number of unique Police Districts: 25
Number of unique Police Beats: 277


In [85]:
"""

So it looks like there was at some point in time a change to a lot of location descriptors which is important to note and be aware of.
This will be something we have to account for. Will wait to see which years we want data for before tackling this because we may be able to
solve this problem by just deleting data before a certain date. 

""" 

'So it looks like there was at some point in time a change to a lot of location descriptors which is important to note \nand be aware of\n'

#CLEANING DATASETS


In [49]:
#POLICE SENTIMENT DATASET CLEANING 

police_sentiment_df.head(5)

,ORG_LEVEL,CITY,AREA,DISTRICT,SECTOR,SAFETY,S_RACE_AFRICAN_AMERICAN,S_RACE_ASIAN_AMERICAN,S_RACE_HISPANIC,S_RACE_WHITE,S_RACE_OTHER,S_AGE_LOW,S_AGE_MEDIUM,S_AGE_HIGH,S_SEX_FEMALE,S_SEX_MALE,S_EDUCATION_LOW,S_EDUCATION_MEDIUM,S_EDUCATION_HIGH,S_INCOME_LOW,S_INCOME_MEDIUM,S_INCOME_HIGH,TRUST,T_RACE_AFRICAN_AMERICAN,T_RACE_ASIAN_AMERICAN,T_RACE_HISPANIC,T_RACE_WHITE,T_RACE_OTHER,T_AGE_LOW,T_AGE_MEDIUM,T_AGE_HIGH,T_SEX_FEMALE,T_SEX_MALE,T_EDUCATION_LOW,T_EDUCATION_MEDIUM,T_EDUCATION_HIGH,T_INCOME_LOW,T_INCOME_MEDIUM,T_INCOME_HIGH,T_LISTEN,T_LISTEN_RACE_AFRICAN_AMERICAN,T_LISTEN_RACE_ASIAN_AMERICAN,T_LISTEN_RACE_HISPANIC,T_LISTEN_RACE_WHITE,T_LISTEN_RACE_OTHER,T_LISTEN_AGE_LOW,T_LISTEN_AGE_MEDIUM,T_LISTEN_AGE_HIGH,T_LISTEN_SEX_FEMALE,T_LISTEN_SEX_MALE,T_LISTEN_EDUCATION_LOW,T_LISTEN_EDUCATION_MEDIUM,T_LISTEN_EDUCATION_HIGH,T_LISTEN_INCOME_LOW,T_LISTEN_INCOME_MEDIUM,T_LISTEN_INCOME_HIGH,T_RESPECT,T_RESPECT_RACE_AFRICAN_AMERICAN,T_RESPECT_RACE_ASIAN_AMERICAN,T_RESPECT_RACE_HISPANIC,T_RESPECT_RACE_WHITE,T_RESPECT_RACE_OTHER,T_RESPECT_AGE_LOW,T_RESPECT_AGE_MEDIUM,T_RESPECT_AGE_HIGH,T_RESPECT_SEX_FEMALE,T_RESPECT_SEX_MALE,T_RESPECT_EDUCATION_LOW,T_RESPECT_EDUCATION_MEDIUM,T_RESPECT_EDUCATION_HIGH,T_RESPECT_INCOME_LOW,T_RESPECT_INCOME_MEDIUM,T_RESPECT_INCOME_HIGH,START_DATE,END_DATE
0,Other,chicago,area_5,14.0,1420.0,56.69,40.27,58.65,61.66,60.06,44.52,48.83,65.43,47.35,65.63,45.56,63.39,58.57,47.30,57.60,56.49,57.74,60.90,57.83,64.16,76.54,40.49,84.10,57.72,53.94,68.25,57.65,65.77,65.63,53.16,72.49,68.69,68.18,44.89,58.82,52.71,62.63,73.52,38.62,81.83,55.60,52.69,66.64,55.97,63.12,63.91,51.84,69.33,68.33,65.39,42.14,62.99,62.96,65.70,79.56,42.37,86.37,59.85,55.18,69.87,59.33,68.41,67.35,54.49,75.65,69.04,70.98,47.64,04/01/2021,04/30/2021
1,Other,chicago,area_2,NaN,NaN,47.31,43.60,42.57,55.28,56.81,57.99,41.20,45.32,52.70,49.32,47.70,44.17,48.85,48.58,48.25,47.45,47.67,58.23,50.62,67.82,64.90,77.99,75.03,64.31,58.87,58.37,60.77,57.79,59.51,55.97,67.49,55.87,57.00,70.80,55.61,47.87,66.53,61.24,76.87,73.40,62.11,56.42,55.99,58.37,54.79,55.87,53.44,65.92,54.42,53.84,66.95,60.84,53.37,69.11,68.56,79.10,76.65,66.51,61.32,60.74,63.17,60.79,63.15,58.50,69.05,57.32,60.17,74.66,04/01/2021,04/30/2021
2,Other,chicago,area_2,5.0,510.0,35.51,32.37,43.24,41.55,48.80,57.43,34.53,34.21,49.43,38.51,36.03,33.98,32.20,43.14,40.45,22.57,51.54,47.52,36.04,67.34,58.50,75.51,73.10,53.69,51.99,51.94,60.90,46.71,49.25,39.71,65.96,42.03,49.46,69.18,44.59,34.34,66.00,52.18,74.94,71.37,49.28,50.82,49.60,55.15,45.16,40.79,38.86,64.12,39.73,46.37,67.23,50.46,37.73,68.69,64.83,76.08,74.84,58.10,53.16,54.28,66.65,48.26,57.72,40.56,67.80,44.34,52.56,71.14,04/01/2021,04/30/2021
3,Other,chicago,area_4,11.0,1110.0,42.63,41.12,41.60,39.17,52.04,61.11,47.86,54.41,27.39,41.09,44.90,42.56,41.88,46.41,38.33,41.77,43.38,46.38,35.31,58.47,54.72,40.01,58.34,50.37,53.67,36.67,46.45,38.51,61.26,36.87,41.59,42.65,49.07,42.79,41.94,30.57,56.98,50.75,38.25,56.08,45.70,51.14,32.10,44.76,30.49,53.88,35.13,38.13,40.89,42.59,37.01,50.82,40.05,59.95,58.69,41.77,60.60,55.04,56.20,41.24,48.13,46.52,68.64,38.61,45.05,44.41,55.56,48.57,04/01/2021,04/30/2021
4,Other,chicago,area_1,9.0,930.0,46.02,37.58,36.90,49.17,50.06,55.76,45.34,38.00,51.73,42.72,44.47,49.92,42.36,37.39,47.72,47.91,33.79,63.57,51.23,63.81,63.01,86.34,54.79,58.43,66.15,68.76,68.63,64.91,57.84,72.11,71.57,60.55,69.56,73.26,59.67,48.34,64.39,57.33,84.45,50.21,53.61,61.62,66.46,64.71,61.77,53.66,69.19,67.65,56.53,66.57,70.05,67.48,54.12,63.24,68.69,88.24,59.37,63.24,70.69,71.07,72.54,68.04,62.02,75.03,75.50,64.57,72.54,76.47,04/01/2021,04/30/2021


In [73]:
police_sentiment_df['AREA'].value_counts()

AREA
area_3     1775
area_1     1491
area_5     1420
area_2     1207
area_4      923
SOUTH        76
NORTH        76
CENTRAL      76
Name: count, dtype: int64

In [103]:
police_sentiment_df['SECTOR'].value_counts()

SECTOR
1420.0    71
2420.0    71
1510.0    71
410.0     71
1230.0    71
2430.0    71
1930.0    71
330.0     71
2230.0    71
2210.0    71
2510.0    71
2520.0    71
230.0     71
1710.0    71
830.0     71
1920.0    71
1530.0    71
910.0     71
2220.0    71
120.0     71
610.0     71
1010.0    71
730.0     71
1430.0    71
630.0     71
1520.0    71
130.0     71
720.0     71
1220.0    71
530.0     71
220.0     71
1020.0    71
2550.0    71
1130.0    71
1210.0    71
510.0     71
1120.0    71
1720.0    71
1410.0    71
1610.0    71
2540.0    71
1830.0    71
920.0     71
430.0     71
2020.0    71
420.0     71
2530.0    71
1810.0    71
1620.0    71
930.0     71
1110.0    71
820.0     71
1030.0    71
1910.0    71
1820.0    71
320.0     71
710.0     71
620.0     71
110.0     71
810.0     71
1650.0    71
1630.0    71
520.0     71
2410.0    71
2010.0    71
2030.0    71
210.0     71
310.0     71
1730.0    71
Name: count, dtype: int64

In [74]:
police_sentiment_df['DISTRICT'].isna().sum()

805

In [42]:
#Remove rows that don't have any data for 'District' since this is the best location connection to the crime data
#Creating a new dataframe
police_df_cleaned = police_sentiment_df.dropna(subset=['DISTRICT'])


In [76]:
#Checking for duplicates
#Finding and displaying duplicate rows
duplicates = police_df_cleaned [police_df_cleaned .duplicated()]
print("Number of duplicate rows:", duplicates.shape[0])

Number of duplicate rows: 0


In [44]:
#Converting dates to datetime 
police_df_cleaned['START_DATE'] = pd.to_datetime(police_df_cleaned['START_DATE'])
police_df_cleaned['END_DATE'] = pd.to_datetime(police_df_cleaned['END_DATE'])

/var/folders/gw/kyktsykx5kddft5zdz_8y75h0000gn/T/ipykernel_23563/4156418273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_df_cleaned['START_DATE'] = pd.to_datetime(police_df_cleaned['START_DATE'])
/var/folders/gw/kyktsykx5kddft5zdz_8y75h0000gn/T/ipykernel_23563/4156418273.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_df_cleaned['END_DATE'] = pd.to_datetime(police_df_cleaned['END_DATE'])


In [45]:
#Removing columns for org_level and city since these are all the same

#Checking if org column has any variations
unique_org_levels = police_df_cleaned['ORG_LEVEL'].unique()
print(unique_org_levels)


# Removing the ORG_LEVEL and CITY columns since there is no variance
police_df_cleaned = police_df_cleaned.drop(columns=['ORG_LEVEL', 'CITY'])

['Other']


In [47]:
#Resetting index 
police_df_cleaned = police_df_cleaned.reset_index(drop=True)

police_df_cleaned.head(5)

,AREA,DISTRICT,SECTOR,SAFETY,S_RACE_AFRICAN_AMERICAN,S_RACE_ASIAN_AMERICAN,S_RACE_HISPANIC,S_RACE_WHITE,S_RACE_OTHER,S_AGE_LOW,S_AGE_MEDIUM,S_AGE_HIGH,S_SEX_FEMALE,S_SEX_MALE,S_EDUCATION_LOW,S_EDUCATION_MEDIUM,S_EDUCATION_HIGH,S_INCOME_LOW,S_INCOME_MEDIUM,S_INCOME_HIGH,TRUST,T_RACE_AFRICAN_AMERICAN,T_RACE_ASIAN_AMERICAN,T_RACE_HISPANIC,T_RACE_WHITE,T_RACE_OTHER,T_AGE_LOW,T_AGE_MEDIUM,T_AGE_HIGH,T_SEX_FEMALE,T_SEX_MALE,T_EDUCATION_LOW,T_EDUCATION_MEDIUM,T_EDUCATION_HIGH,T_INCOME_LOW,T_INCOME_MEDIUM,T_INCOME_HIGH,T_LISTEN,T_LISTEN_RACE_AFRICAN_AMERICAN,T_LISTEN_RACE_ASIAN_AMERICAN,T_LISTEN_RACE_HISPANIC,T_LISTEN_RACE_WHITE,T_LISTEN_RACE_OTHER,T_LISTEN_AGE_LOW,T_LISTEN_AGE_MEDIUM,T_LISTEN_AGE_HIGH,T_LISTEN_SEX_FEMALE,T_LISTEN_SEX_MALE,T_LISTEN_EDUCATION_LOW,T_LISTEN_EDUCATION_MEDIUM,T_LISTEN_EDUCATION_HIGH,T_LISTEN_INCOME_LOW,T_LISTEN_INCOME_MEDIUM,T_LISTEN_INCOME_HIGH,T_RESPECT,T_RESPECT_RACE_AFRICAN_AMERICAN,T_RESPECT_RACE_ASIAN_AMERICAN,T_RESPECT_RACE_HISPANIC,T_RESPECT_RACE_WHITE,T_RESPECT_RACE_OTHER,T_RESPECT_AGE_LOW,T_RESPECT_AGE_MEDIUM,T_RESPECT_AGE_HIGH,T_RESPECT_SEX_FEMALE,T_RESPECT_SEX_MALE,T_RESPECT_EDUCATION_LOW,T_RESPECT_EDUCATION_MEDIUM,T_RESPECT_EDUCATION_HIGH,T_RESPECT_INCOME_LOW,T_RESPECT_INCOME_MEDIUM,T_RESPECT_INCOME_HIGH,START_DATE,END_DATE
0,area_5,14.0,1420.0,56.69,40.27,58.65,61.66,60.06,44.52,48.83,65.43,47.35,65.63,45.56,63.39,58.57,47.30,57.60,56.49,57.74,60.90,57.83,64.16,76.54,40.49,84.10,57.72,53.94,68.25,57.65,65.77,65.63,53.16,72.49,68.69,68.18,44.89,58.82,52.71,62.63,73.52,38.62,81.83,55.60,52.69,66.64,55.97,63.12,63.91,51.84,69.33,68.33,65.39,42.14,62.99,62.96,65.70,79.56,42.37,86.37,59.85,55.18,69.87,59.33,68.41,67.35,54.49,75.65,69.04,70.98,47.64,2021-04-01,2021-04-30
1,area_2,5.0,510.0,35.51,32.37,43.24,41.55,48.80,57.43,34.53,34.21,49.43,38.51,36.03,33.98,32.20,43.14,40.45,22.57,51.54,47.52,36.04,67.34,58.50,75.51,73.10,53.69,51.99,51.94,60.90,46.71,49.25,39.71,65.96,42.03,49.46,69.18,44.59,34.34,66.00,52.18,74.94,71.37,49.28,50.82,49.60,55.15,45.16,40.79,38.86,64.12,39.73,46.37,67.23,50.46,37.73,68.69,64.83,76.08,74.84,58.10,53.16,54.28,66.65,48.26,57.72,40.56,67.80,44.34,52.56,71.14,2021-04-01,2021-04-30
2,area_4,11.0,1110.0,42.63,41.12,41.60,39.17,52.04,61.11,47.86,54.41,27.39,41.09,44.90,42.56,41.88,46.41,38.33,41.77,43.38,46.38,35.31,58.47,54.72,40.01,58.34,50.37,53.67,36.67,46.45,38.51,61.26,36.87,41.59,42.65,49.07,42.79,41.94,30.57,56.98,50.75,38.25,56.08,45.70,51.14,32.10,44.76,30.49,53.88,35.13,38.13,40.89,42.59,37.01,50.82,40.05,59.95,58.69,41.77,60.60,55.04,56.20,41.24,48.13,46.52,68.64,38.61,45.05,44.41,55.56,48.57,2021-04-01,2021-04-30
3,area_1,9.0,930.0,46.02,37.58,36.90,49.17,50.06,55.76,45.34,38.00,51.73,42.72,44.47,49.92,42.36,37.39,47.72,47.91,33.79,63.57,51.23,63.81,63.01,86.34,54.79,58.43,66.15,68.76,68.63,64.91,57.84,72.11,71.57,60.55,69.56,73.26,59.67,48.34,64.39,57.33,84.45,50.21,53.61,61.62,66.46,64.71,61.77,53.66,69.19,67.65,56.53,66.57,70.05,67.48,54.12,63.24,68.69,88.24,59.37,63.24,70.69,71.07,72.54,68.04,62.02,75.03,75.50,64.57,72.54,76.47,2021-04-01,2021-04-30
4,area_5,16.0,1620.0,55.97,57.84,51.32,59.89,53.82,48.03,62.47,54.67,52.92,55.61,55.31,60.94,55.75,48.57,57.74,51.88,63.00,79.77,72.23,85.09,82.14,77.07,92.10,88.24,78.13,79.14,76.45,82.68,80.18,79.26,77.36,78.38,79.16,79.19,78.90,69.03,83.93,81.75,75.75,91.76,89.18,77.58,77.56,75.52,81.66,79.72,78.23,75.86,76.23,78.77,78.12,80.65,75.43,86.26,82.53,78.40,92.43,87.31,78.69,80.72,77.37,83.70,80.64,80.29,78.86,80.53,79.55,80.26,2021-04-01,2021-04-30


In [53]:
#GROCERY STORE DATASET CLEANING 
grocery_stores_df.head(5)

,STORE NAME,LICENSE ID,ACCOUNT NUMBER,SQUARE FEET,BUFFER SIZE,ADDRESS,ZIP CODE,COMMUNITY AREA NAME,COMMUNITY AREA,WARD,CENSUS TRACT,CENSUS BLOCK,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,BOB'S LIQUORS & GROCERIES,957.0,86492.0,10001.0,A,5069 W MADISON ST,60644,AUSTIN,25,28,17031252201,1.703125e+14,1.142425e+06,1.899505e+06,41.880296,-87.752494,"(41.8802960895, -87.7524941982)"
1,QUICK STOP FOOD MART,39791.0,3604.0,3000.0,B,2751 W PRATT BLVD,60645,WEST RIDGE,2,50,17031020601,1.703102e+14,1.156543e+06,1.944959e+06,42.004750,-87.699423,"(42.0047500731, -87.6994227766)"
2,DOMINICK'S #147/1147,41674.0,5033.0,68000.0,A,1340 S CANAL ST,60607,NEAR WEST SIDE,28,2,17031841900,1.703184e+14,1.173296e+06,1.893986e+06,41.864521,-87.639304,"(41.8645211937, -87.6393040996)"
3,CERMAK PRODUCE,1515206.0,279468.0,25000.0,A,5220 S PULASKI RD,60632,WEST ELSDON,62,23,17031620200,1.703162e+14,1.150549e+06,1.869569e+06,41.797992,-87.723444,"(41.79799187, -87.7234437408)"
4,WHOLE FOODS MARKET,54059.0,86298.0,4501.0,B,3300 N ASHLAND AVE,60657,LAKE VIEW,6,44,17031062500,1.703106e+14,1.165028e+06,1.921997e+06,41.941566,-87.668860,"(41.9415658578, -87.6688604029)"


In [51]:
#Checking for duplicates
#Finding and displaying duplicate rows
duplicates = grocery_stores_df [grocery_stores_df.duplicated()]
print("Number of duplicate rows:", duplicates.shape[0])

Number of duplicate rows: 0


In [ ]:
#Dataset looks good I don't think there is anything to remove from the dataset at this time

In [118]:
#BUS AND TRAIN RIDERSHIP DATASET CLEANING 

bus_df.tail(5)

,route,routename,Month_Beginning,Avg_Weekday_Rides,Avg_Saturday_Rides,Avg_Sunday-Holiday_Rides,MonthTotal
36720,172,U. of Chicago/Kenwood,06/01/2023,696.6,63.0,0.0,15452
36721,192,U. of Chicago Hospitals Express,06/01/2023,350.0,0.0,0.0,7701
36722,201,Central/Ridge,06/01/2023,1287.4,749.4,0.0,31319
36723,206,Evanston Circulator,06/01/2023,136.8,0.0,0.0,3009
36724,1001,Shuttle/Special Event Route,06/01/2023,56.6,58.4,106.8,1906


In [120]:
unique_route_ids = bus_df['route'].nunique()
unique_routename = bus_df['routename'].nunique()

print(unique_route_ids)
print(unique_routename)

184
194


In [121]:
unique_route_ids = bus_df['route'].unique()
unique_routename = bus_df['routename'].unique()

print(unique_route_ids)
print(unique_routename)

['1' '2' '3' '4' '6' '7' '8' '8A' '9' '10' '11' '12' '14' '17' '18' '19'
 '20' '21' 'X21' '22' '24' '25' '27' '28' '29' '30' '33' '34' '35' '36'
 '37' '39' '43' '44' '47' '48' '49' '49A' '49B' 'X49' '50' '51' '52' '52A'
 '53' '53A' '54' '54A' '54B' '55' '55N' '56' '56A' '57' '59' '60' '62'
 '62H' '63' '63W' '64' '65' '66' '67' '68' '69' '70' '71' '72' '73' '74'
 '75' '76' '77' '78' '79' '80' '81' '81W' '82' '84' '85' '85A' '86' '87'
 '88' '90' '90N' '91' '92' '93' '94' '95E' '95W' '96' '97' '100' '103'
 '106' '108' '111' '112' '119' '120' '121' '122' '123' '125' '126' '127'
 '129' '135' '136' '145' '146' '147' '151' '152' '155' '156' '157' '169'
 '170' '171' '172' '173' '201' '202' '203' '204' '128' '124' '154' '130'
 '31' '992' '168' 'X80' 'X98' 'X55' '69BR' '5' '200' '205' '206' '15' '26'
 '134' '143' '144' '1001' '148' '40' 'X99' 'X28' '53AL' 'X3' 'X4' 'X54'
 '192' '1002' 'X9' 'X20' '38' '174' '165' '55A' '132' '290' 'N201' '115'
 'J14' 'R22' 'R55' 'R63' 'R69' 'R79' 'R87' 'R95' 'R39

In [116]:
train_ridership_df.head(5)

,station_id,stationame,month_beginning,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
0,40900,Howard,01/01/2001,6233.9,3814.5,2408.6,164447
1,41190,Jarvis,01/01/2001,1489.1,1054.0,718.0,40567
2,40100,Morse,01/01/2001,4412.5,3064.5,2087.8,119772
3,41300,Loyola,01/01/2001,4664.5,3156.0,1952.8,125008
4,40760,Granville,01/01/2001,3109.8,2126.0,1453.8,84189


In [72]:
cta_ridership_df['station_id'].nunique()

147

In [71]:
cta_ridership_df['stationame'].nunique()

148

In [82]:
#Odd that these two columns are off by one. Seems like each station name should have exactly one station_id but there is an error in the data
#Checking to see which station id has two station names associated with it 

# Group by station_id and count unique station_names
unique_names_per_id = cta_ridership_df.groupby('station_id')['stationame'].nunique()

# Find station_ids with more than one unique station_name
mismatched_ids = unique_names_per_id[unique_names_per_id > 1]

print("Station IDs with mismatched names:", mismatched_ids)

#Not really sure how would go about correcting this. Just good to know there is likely an error with this dataset.
#Probably best to go with station name instead of id d/t this error. 


#TODO - if we want to use this data we will HAVE TO convert stationames to some other location like community areas. 
#Same is true for bus stops

Station IDs with mismatched names: station_id
40140    2
Name: stationame, dtype: int64


In [94]:
#CLEANING CENSUS DATASET
census_df.head(77)

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0
5,6.0,Lake View,1.1,11.4,4.7,2.6,17.0,60058,5.0
6,7.0,Lincoln Park,0.8,12.3,5.1,3.6,21.5,71551,2.0
7,8.0,Near North Side,1.9,12.9,7.0,2.5,22.6,88669,1.0
8,9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0
9,10.0,Norwood Park,2.0,5.4,9.0,11.5,39.5,32875,21.0


In [102]:
#CLEANING HOUSING DATASET

housing_df.head(5)

,Community Area Name,Community Area Number,Property Type,Property Name,Address,Zip Code,Phone Number,Management Company,Units,X Coordinate,Y Coordinate,Latitude,Longitude,Location
0,Near West Side,28,ARO,The Dylan,160 N. Morgan St.,60607,872.328.0028,Sterling Bay,28,1169706.814,1901381.768,41.884895,-87.652265,"(41.884895408116236, -87.65226488412628)"
1,Hegewisch,55,Senior,Senior Suites of Hegewisch,13550 S. Avenue O,60633,773-646-1333,Senior Lifestyle Corp.,116,1201038.059,1815487.733,41.648457,-87.540123,"(41.648457411436, -87.5401231660406)"
2,Uptown,3,Senior,Wilson Yard Senior Apts.,1032 W. Montrose Ave,60613,866-227-0438,Holsten Real Estate Dev. Corp.,13,1168438.959,1929421.650,41.961866,-87.656107,"(41.9618658532981, -87.656107428372)"
3,Lake View,6,ARO,The Mode,3462 N. Lincoln Ave.,60657,773-205-8300,606 Realty,2,1164033.827,1923207.502,41.944908,-87.672479,"(41.9449083304908, -87.6724793376448)"
4,North Lawndale,29,Multifamily,Lawndale Restoration II Apts. (scattered sites),10 N. Hamlin Ave.,60613,312-337-5339,Holsten Management Corp.,170,1151024.342,1899855.056,41.881092,-87.720910,"(41.8810923305496, -87.7209100899284)"


In [ ]:
# Sorting the DataFrame by community area number
housing_df_sorted = housing_df.sort_values(by='Community Area Number')

# Resetting the index
housing_df_sorted.reset_index(drop=True, inplace=True)

In [99]:
housing_df_sorted.head(80)

,Community Area Name,Community Area Number,Property Type,Property Name,Address,Zip Code,Phone Number,Management Company,Units,X Coordinate,Y Coordinate,Latitude,Longitude,Location
0,Rogers Park,1,Senior,Levy House,1221 W. Sherwin Ave.,60626,773-508-1000,Preservation Housing Management LLC,57,1.166287e+06,1.948743e+06,42.014932,-87.663463,"(42.01493175019485, -87.66346311838282)"
1,Rogers Park,1,Multifamily,Clark Estes Apts.,1763 W. Estes Ave.,60626,773-293-7443,"Ross Financial Services, Inc.",54,1.163232e+06,1.947395e+06,42.011298,-87.674743,"(42.011297553855016, -87.67474340393993)"
2,Rogers Park,1,Supportive Housing,Wayne Street Apts.,6808 N. Wayne Ave.,60626,773-572-5272,The Thresholds,297,1.165865e+06,1.945402e+06,42.005771,-87.665111,"(42.0057709728472, -87.6651110903791)"
3,Rogers Park,1,Multifamily,The Concord at Sheridan,6438 N. Sheridan Road,60626,773-856-0451,The Habitat Co.,57,1.167067e+06,1.942855e+06,41.998758,-87.660763,"(41.99875752497431, -87.66076275537927)"
4,Rogers Park,1,Senior,James Sneider Apts.,7450 N. Rogers Ave.,60626,773-743-3699,Hispanic Housing Dev. Corp.,57,1.163641e+06,1.949531e+06,42.017150,-87.673179,"(42.0171497110557, -87.6731785733257)"
5,Rogers Park,1,Inter-generational,Nathalie Salmon House,7320 N. Sheridan Road,60626,773-921-3200,H.O.M.E.,54,1.166012e+06,1.948540e+06,42.014380,-87.664483,"(42.01438027189406, -87.6644826893636)"
6,Rogers Park,1,Senior,Caroline Hedger Apts.,6400 N. Sheridan Road,60626,773-743-7273,The CHA,436,1.167087e+06,1.942659e+06,41.998218,-87.660695,"(41.99821792053241, -87.6606952555721)"
7,Rogers Park,1,Senior,Morse Senior Apts.,1528 W. Morse Ave.,60626,312-602-6207,CIG Management,44,1.165844e+06,1.946059e+06,42.007574,-87.665171,"(42.0075737709331, -87.6651711448293)"
8,Rogers Park,1,ARO,1730 W. Greenleaf,1730 W. Greenleaf Ave.,60626,773-605-0103,DLG Management,3,1.163535e+06,1.947041e+06,42.010320,-87.673637,"(42.0103197756509, -87.6736370200105)"
9,Rogers Park,1,Senior,Wayne Senior Housing,6928 N. Wayne Ave.,60626,773-274-0177,CIG Management,119,1.165844e+06,1.946059e+06,42.007574,-87.665171,"(42.0075737709331, -87.6651711448293)"


In [ ]:
#Save clean datasets as new csv's
